In [11]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os

# Base directory where all patients' folders are stored
base_dir = '/Volumes/Mahdi-2022/Laxman_PD_Project/MAP_complete'

# Output directory for the Excel files
output_dir = '/Users/christianraimondo/Desktop/ML-PD-Revamp/Registration-Networks/'

patient_names = [
    
]


# Generate a list of patient directories
patient_dirs = [os.path.join(base_dir, name) for name in patient_names]

# List of diffusion images to process
dti_images = ['FA.nii.gz', 'MD.nii.gz', 'AD.nii.gz', 'PA.nii', 'RTAP.nii', 'RTOP.nii', 'RTPP.nii']

# ROI atlas files
roi_atlases = ['r_motor_ROI.nii', 'r_networks_ROI.nii', 'r_visual_ROI.nii']

# Function to process images for a given patient and ROI
def process_images(patient_dir, roi_file):
    patient_results = []
    for img_file in dti_images:
        image_path = os.path.join(patient_dir, 'dti_outputs', img_file)
        label_path = os.path.join(patient_dir, 'dti_outputs', roi_file)
        
        if not os.path.exists(image_path) or not os.path.exists(label_path):
            continue  # Skip if the necessary files are not found
        
        # Read the image and label map
        image = sitk.ReadImage(image_path)
        label_map = sitk.ReadImage(label_path)
        label_array = sitk.GetArrayFromImage(label_map)
        image_array = sitk.GetArrayFromImage(image)
        
        unique_labels = np.unique(label_array)
        
        for label in unique_labels:
            if label == 0:
                continue  # Skip background
            
            label_mask = label_array == label
            labeled_voxels = image_array[label_mask]
            
            # Calculate the statistics
            mean_intensity = labeled_voxels.mean() if labeled_voxels.size > 0 else 0
            std_deviation = labeled_voxels.std() if labeled_voxels.size > 0 else 0
            voxel_count = labeled_voxels.size
            
            patient_results.append({
                'Patient': os.path.basename(patient_dir),
                'ImageType': img_file.replace('.nii.gz', '').replace('.nii', ''),
                'ROI': roi_file,
                'Label': int(label),  # Store label as an integer for sorting
                'MeanIntensity': mean_intensity,
                'StdDeviation': std_deviation,
                'VoxelCount': voxel_count
            })
    return patient_results

# Process each ROI atlas
results_by_roi = {roi: [] for roi in roi_atlases}
for patient_dir in patient_dirs:
    for roi in roi_atlases:
        results_by_roi[roi].extend(process_images(patient_dir, roi))

# Export to Excel with specific structure
for roi, results in results_by_roi.items():
    df_all_results = pd.DataFrame(results)
    # Splitting patient names into first and last name
    df_all_results[['Last Name', 'First Name']] = df_all_results['Patient'].str.split('_', n=1, expand=True)

    
    excel_filename = os.path.join(output_dir, f'patient_statistics_{roi}.xlsx')
    with pd.ExcelWriter(excel_filename) as writer:
        for image in dti_images:
            image_clean = image.replace('.nii.gz', '').replace('.nii', '')
            df_image = df_all_results[df_all_results['ImageType'] == image_clean]
            if not df_image.empty:
                pivot_table = df_image.pivot_table(
                    index=['Patient', 'Last Name', 'First Name'],
                    columns='Label',
                    values=['VoxelCount', 'MeanIntensity', 'StdDeviation'],
                    aggfunc='first'
                )
                # Sort columns numerically based on label and then reformat the column names
                pivot_table = pivot_table.reindex(sorted(pivot_table.columns), axis=1)  # Sort by label number
                pivot_table.columns = [f"{image_clean}_{stat}_{label}" for (label, stat) in pivot_table.columns]
                pivot_table.to_excel(writer, sheet_name=image_clean)
